In [1]:
import quandl
import datetime as dt
import pandas as pd
import numpy as np

In [2]:
df = quandl.get('YAHOO/INDEX_DJI')

In [3]:
df.columns = [x.lower() for x in df.columns]

In [4]:
df_dow_jones = df.copy()

In [5]:
# add weekday number (4 == Friday)
df_weekday = pd.DataFrame(df_dow_jones.index.map(lambda x: x.weekday), index = df_dow_jones.index)
df_weekday.columns = ['weekday']
                          
df_dow_jones = df_dow_jones.merge(df_weekday, how='outer', left_index=True, right_index=True)

In [6]:
# add week number
df_week_number = pd.DataFrame(df_dow_jones.index.map(lambda x: x.week - 1), index=df_dow_jones.index)
df_week_number.columns = ['week_number']
df_dow_jones = df_dow_jones.merge(df_week_number, how='outer', left_index=True, right_index=True)

In [7]:
# add year

df_year = pd.DataFrame(df_dow_jones.index.map(lambda x: x.year), index=df_dow_jones.index)
df_year.columns = ['year']
df_dow_jones = df_dow_jones.merge(df_year, how='outer', left_index=True, right_index=True)

In [8]:
df_dj_friday = df_dow_jones.loc[df_dow_jones['weekday'] == 4].copy()

In [9]:
df_dj_friday['week_shift'] = df_dj_friday['week_number'] - df_dj_friday['week_number'].shift(1)

In [10]:
def get_df_year_week():
    # get 1985 - start week 13 (index 0)
    list_year_week = [(1985, i) for i in range(13, dt.datetime(1985, 12, 28).isocalendar()[1])]

    def create_list_year_week(year):
        return [(year, i) for i in range(dt.datetime(year, 12, 28).isocalendar()[1])]

    present_year = dt.datetime.now().year
    for i in range(1986, present_year):
        list_year_week.extend(create_list_year_week(i))

    # present year to this week
    list_year_week.extend([(present_year, i) for i in range(dt.datetime.now().isocalendar()[1])])

    return pd.DataFrame(list_year_week, columns=['year', 'week_number'])

In [11]:
df_year_week = get_df_year_week()

In [14]:
def get_end_of_week_values(df_to, df_from, week_day):
    df_to_ = df_to.copy()
    df_from_ = df_from.copy()
    if week_day < 0:
        return df_to_
    if 'weekday' not in df_to_.columns:
        df_new_values = df_to_.merge(df_from_.loc[df_from_['weekday'] == week_day], how='left', left_on=['year', 'week_number'], right_on=['year', 'week_number'])
        df_to_['weekday'] = np.nan
    else:
        df_new_values = df_to_.loc[df_to_['weekday'].isnull(), ['year', 'week_number']].merge(df_from_.loc[df_from_['weekday'] == week_day], how='left', left_on=['year', 'week_number'], right_on=['year', 'week_number'])
    df_to_.drop(df_to.loc[df_to_['weekday'].isnull()].index, inplace=True)
    df_to_ = df_to_.append(df_new_values)
    if df_to_.loc[df_to_['weekday'].isnull()].shape[0] == 0:
        return df_to_
    else:
        return get_end_of_week_values(df_to_, df_from_, week_day - 1)

In [17]:
df_end_of_week_values = get_end_of_week_values(df_year_week, df_dow_jones, 4)
df_end_of_week_values = df_end_of_week_values.sort_values(['year', 'week_number']).reset_index(drop=True)

In [18]:
df_end_of_week_values

,adjusted close,close,high,low,open,volume,week_number,weekday,year
0,1265.680054,1265.680054,1270.430054,1257.290039,1263.689941,7320000.0,14,4.0,1985
1,1266.560059,1266.560059,1272.530029,1258.500000,1265.130005,8740000.0,15,4.0,1985
2,1275.180054,1275.180054,1290.300049,1270.540039,1286.219971,8930000.0,16,4.0,1985
3,1247.239990,1247.239990,1253.859985,1238.300049,1243.709961,8680000.0,17,4.0,1985
4,1274.180054,1274.180054,1282.459961,1261.589966,1268.989990,16460000.0,18,4.0,1985
5,1285.339966,1285.339966,1295.829956,1271.420044,1276.829956,14230000.0,19,4.0,1985
6,1301.969971,1301.969971,1305.670044,1291.109985,1295.469971,6680000.0,20,4.0,1985
7,1315.410034,1315.410034,1320.790039,1298.719971,1302.760010,12190000.0,21,4.0,1985
8,1316.420044,1316.420044,1331.989990,1312.949951,1329.640015,8590000.0,22,4.0,1985
9,1300.959961,1300.959961,1305.439941,1288.880005,1297.380005,9220000.0,23,4.0,1985
